In [1]:
from pathlib import Path
import json
DATA_PATH=Path("../data")
jobpost_file_path:Path = DATA_PATH.joinpath("job_post_list_20250901.json")
assert jobpost_file_path.exists()

job_post_data = [] #สร้าง list ขึ้นมา และอ่านข้อมูลแต่ละแถวใน json เก็บเป็น dict แล้วเก็บหลายๆ dict เป็น list รวมเป็น dataframe ขึ้นมา
with open(jobpost_file_path, "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()
        if line:  # skip empty lines
            job_post_data.append(json.loads(line))

In [2]:
len(job_post_data), job_post_data[0]

(37142,
 {'job_id': '86122800',
  'url': 'https://th.jobsdb.com/job/86122800?type=standard&ref=search-standalone#sol=920eb283fcb9807e8b675bd116b40575fbe3599f',
  'extraction_date': '2025-08-10',
  'job_title': 'Management Trainee Account',
  'company': 'BIRLA CARBON (THAILAND) PUBLIC COMPANY LIMITED',
  'location': 'Ang Thong',
  'job_function': 'Accounting',
  'work_type': 'Full time',
  'salary': '',
  'description': "Job Description:\n\nFixed Assets Accounting for all 3 Units\n\nFixed asset Physical verification\n\nComplete physical verification at plant.\n\nUpdating of cycle count data to SAP.\n\nReconcile cycle count with SAP after verification.\n\nEmployee Payroll for local & expats\n\nWHT for personal taxes\n\nWHT working preparation reconciling with GL's & advising monthly payment.\n\nChecking and return filing on monthly and yearly basis (PND1 etc.)\n\nAudit Requirement\n\nProviding necessary documents related to Payroll such as register, headcount and other relevant details f

In [3]:
from models import JobPost
JobPost(**job_post_data[0])

JobPost(job_id='86122800', url='https://th.jobsdb.com/job/86122800?type=standard&ref=search-standalone#sol=920eb283fcb9807e8b675bd116b40575fbe3599f', extraction_date=datetime.date(2025, 8, 10), job_title='Management Trainee Account', company='BIRLA CARBON (THAILAND) PUBLIC COMPANY LIMITED', location='Ang Thong', job_function='Accounting', work_type='Full time', salary='', description="Job Description:\n\nFixed Assets Accounting for all 3 Units\n\nFixed asset Physical verification\n\nComplete physical verification at plant.\n\nUpdating of cycle count data to SAP.\n\nReconcile cycle count with SAP after verification.\n\nEmployee Payroll for local & expats\n\nWHT for personal taxes\n\nWHT working preparation reconciling with GL's & advising monthly payment.\n\nChecking and return filing on monthly and yearly basis (PND1 etc.)\n\nAudit Requirement\n\nProviding necessary documents related to Payroll such as register, headcount and other relevant details for quarterly statutory audit.\n\nPre

In [4]:
job_posts:list[JobPost] = []
for jp in job_post_data:
    job_post = JobPost(**jp)
    job_posts.append(job_post)
    # print(job_post.postedAt, type(job_post.postedAt))

In [5]:
job_posts[0]

JobPost(job_id='86122800', url='https://th.jobsdb.com/job/86122800?type=standard&ref=search-standalone#sol=920eb283fcb9807e8b675bd116b40575fbe3599f', extraction_date=datetime.date(2025, 8, 10), job_title='Management Trainee Account', company='BIRLA CARBON (THAILAND) PUBLIC COMPANY LIMITED', location='Ang Thong', job_function='Accounting', work_type='Full time', salary='', description="Job Description:\n\nFixed Assets Accounting for all 3 Units\n\nFixed asset Physical verification\n\nComplete physical verification at plant.\n\nUpdating of cycle count data to SAP.\n\nReconcile cycle count with SAP after verification.\n\nEmployee Payroll for local & expats\n\nWHT for personal taxes\n\nWHT working preparation reconciling with GL's & advising monthly payment.\n\nChecking and return filing on monthly and yearly basis (PND1 etc.)\n\nAudit Requirement\n\nProviding necessary documents related to Payroll such as register, headcount and other relevant details for quarterly statutory audit.\n\nPre

## Create Table in BigQuery for JobPost

In [6]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.envfiles/dev.env")
import os

BQ_PROJECT = os.environ["BQ_PROJECT"]
BQ_DATASET = os.environ["BQ_DATASET"]

from db import get_client, dataset_id, get_dataset
client = get_client()
# dataset = get_dataset()

In [7]:
JobPost.delete_table(dataset_id, client)

JobPost.create_table(dataset_id, client)

Deleted table 'JobPosts'.
Created table skill-master.dev.JobPosts


In [8]:
for job_post in job_posts[:100]:
    print(job_post)
    job_post.insert(dataset_id, client)
    

job_id='86122800' url='https://th.jobsdb.com/job/86122800?type=standard&ref=search-standalone#sol=920eb283fcb9807e8b675bd116b40575fbe3599f' extraction_date=datetime.date(2025, 8, 10) job_title='Management Trainee Account' company='BIRLA CARBON (THAILAND) PUBLIC COMPANY LIMITED' location='Ang Thong' job_function='Accounting' work_type='Full time' salary='' description="Job Description:\n\nFixed Assets Accounting for all 3 Units\n\nFixed asset Physical verification\n\nComplete physical verification at plant.\n\nUpdating of cycle count data to SAP.\n\nReconcile cycle count with SAP after verification.\n\nEmployee Payroll for local & expats\n\nWHT for personal taxes\n\nWHT working preparation reconciling with GL's & advising monthly payment.\n\nChecking and return filing on monthly and yearly basis (PND1 etc.)\n\nAudit Requirement\n\nProviding necessary documents related to Payroll such as register, headcount and other relevant details for quarterly statutory audit.\n\nPreparation of Direc

In [9]:
rows = JobPost.fetch_all(dataset_id, client)
len(rows)


100

In [10]:
rows[0]

JobPost(job_id='86122800', url='https://th.jobsdb.com/job/86122800?type=standard&ref=search-standalone#sol=920eb283fcb9807e8b675bd116b40575fbe3599f', extraction_date=datetime.date(2025, 8, 10), job_title='Management Trainee Account', company='BIRLA CARBON (THAILAND) PUBLIC COMPANY LIMITED', location='Ang Thong', job_function='Accounting', work_type='Full time', salary='', description="Job Description:\n\nFixed Assets Accounting for all 3 Units\n\nFixed asset Physical verification\n\nComplete physical verification at plant.\n\nUpdating of cycle count data to SAP.\n\nReconcile cycle count with SAP after verification.\n\nEmployee Payroll for local & expats\n\nWHT for personal taxes\n\nWHT working preparation reconciling with GL's & advising monthly payment.\n\nChecking and return filing on monthly and yearly basis (PND1 etc.)\n\nAudit Requirement\n\nProviding necessary documents related to Payroll such as register, headcount and other relevant details for quarterly statutory audit.\n\nPre